In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import pandas as pd 
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score

In [2]:
df_processed = pd.read_excel("Pre_processed_data.xlsx")

In [3]:
df_processed.head(5)

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Out_age,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,...,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Identifier_2_DR,Item_Identifier_2_FD,Item_Identifier_2_NC,Outlet_Identifier,Item_Identifier,source
0,9.30,0.016047,249.8092,3735.1380,14,1,0,0,0,0,...,0,1,0,0,0,1,0,OUT049,FDA15,train
1,5.92,0.019278,48.2692,443.4228,4,0,1,0,0,0,...,0,0,1,0,1,0,0,OUT018,DRC01,train
2,17.50,0.016760,141.6180,2097.2700,14,1,0,0,0,0,...,0,1,0,0,0,1,0,OUT049,FDN15,train
3,19.20,0.072558,182.0950,732.3800,15,0,1,0,0,0,...,1,0,0,0,0,1,0,OUT010,FDX07,train
4,8.93,0.062099,53.8614,994.7052,26,1,0,0,0,0,...,0,1,0,0,0,0,1,OUT013,NCD19,train


In [4]:
train_preprocessed = df_processed.loc[df_processed['source']=='train']
test_preprocessed = df_processed.loc[df_processed['source']=='test']

In [5]:
train_preprocessed.shape,test_preprocessed.shape

((8523, 36), (5681, 36))

In [6]:
del train_preprocessed['source']

In [7]:
del test_preprocessed['source']

In [8]:
train_preprocessed.head(5)

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Out_age,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,...,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Identifier_2_DR,Item_Identifier_2_FD,Item_Identifier_2_NC,Outlet_Identifier,Item_Identifier
0,9.30,0.016047,249.8092,3735.1380,14,1,0,0,0,0,...,0,0,1,0,0,0,1,0,OUT049,FDA15
1,5.92,0.019278,48.2692,443.4228,4,0,1,0,0,0,...,1,0,0,1,0,1,0,0,OUT018,DRC01
2,17.50,0.016760,141.6180,2097.2700,14,1,0,0,0,0,...,0,0,1,0,0,0,1,0,OUT049,FDN15
3,19.20,0.072558,182.0950,732.3800,15,0,1,0,0,0,...,1,1,0,0,0,0,1,0,OUT010,FDX07
4,8.93,0.062099,53.8614,994.7052,26,1,0,0,0,0,...,1,0,1,0,0,0,0,1,OUT013,NCD19


In [9]:
X_train = train_preprocessed.drop(columns = ['Outlet_Identifier','Item_Outlet_Sales','Item_Identifier'])
X_test = test_preprocessed.drop(columns = ['Outlet_Identifier','Item_Outlet_Sales','Item_Identifier'])
train_y = train_preprocessed['Item_Outlet_Sales']
test_y = test_preprocessed['Item_Outlet_Sales']

In [10]:
#R2 - 1 Golden Standard - Expert/Target included in Training
#R2- SSR/SST - R2

In [11]:
lr = LinearRegression()
lr.fit(X_train,train_y)
pred_train = lr.predict(X_train)
pred_test = lr.predict(X_test)
pred_test = abs(pred_test)
print("MAE: ",mean_absolute_error(train_y,pred_train))
print("MSE: ",mean_squared_error(train_y,pred_train))
print("RMSE: ",np.sqrt(mean_squared_error(train_y,pred_train)))
print("R-Squared: ",r2_score(train_y,pred_train))

MAE:  836.0540595865735
MSE:  1273442.6123655152
RMSE:  1128.4691455088682
R-Squared:  0.5626612621848155


In [16]:
#cross Validation starts
lr = LinearRegression()
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lr,X_train,train_y,scoring ='r2',cv=folds)
scores

array([0.54760906, 0.56691642, 0.55469593, 0.55976688, 0.57254733])

In [20]:
#RMSE 
lr = LinearRegression()
#folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(lr,X_train,train_y,scoring ='neg_mean_squared_error',cv=folds)
score_RMSE = [np.sqrt(i) for i in abs(scores)]
np.mean(score_RMSE)

1131.2183655103158

In [21]:
score_RMSE

[1155.2915552944576,
 1086.0930009702727,
 1154.4112302682306,
 1139.4678921047002,
 1120.8281489139192]

In [ ]:
#Stronger report of model performance on K different train and Val sets 